# Estudio de funcionamiento de EDVR, herramienta de superresolución de vídeos
## TFG del Grado en Ingeniería Informática en la Universidad de Burgos. Curso 2020-21

Realizado por:
*  Gonzalo Murillo Montes 

Tutores:
*  Pedro Latorre Carmona
*  César Ignacio Garcıa Osorio

```
@misc{wang2020basicsr,
  author =       {Xintao Wang and Ke Yu and Kelvin C.K. Chan and
                  Chao Dong and Chen Change Loy},
  title =        {BasicSR},
  howpublished = {\url{https://github.com/xinntao/BasicSR}},
  year =         {2020}
}
```
Xintao Wang, Kelvin C.K. Chan, Ke Yu, Chao Dong, &amp; Chen Change Loy. (2019, May 7). EDVR: Video Restoration with Enhanced Deformable Convolutional Networks [PDF]. Ithaca, New York: Cornell University.

###  ¿NTIRE19 Challenge?

New Trends in Image Restoration and Enhancement workshop and challenges on image and video restoration and enhancement.
Proponen diversos retos en las áreas de la restauración de imágenes y vídeos y los participntes compiten para obtener los mejores resultados.

EDVR es la solución ganadora del NTIRE19 Challenge en la categoría de super-resolucion de vídeos, centrada en dos aspectos:

* Alienación de múltiples frames con mucho movimiento.
* Fusion de frames con distintos movimientos y partes borrosas. 

EDVR ganó en las 4 pruebas relacionadas con vídeos con amplio margen. 

###  ¿Qué es EDVR?

EDVR es el acrónimo de Enhanced Deformable convolutions Video Restoration, es una herramienta conteniada en BasicSR, centrada en la super-resolución y desemborronamiento de vídeos.

Está basada en dos principios el PCD (Pyramid,
Cascading and Deformable convolutions) y el TSA (Temporal and Spatial Attention)

* PCD: usa convoluciones deformables para agrupar píxeles cercanos, usando una estructura piramidal invertida empezando a pequeña escala y progresivamente aumentando, junto a esto se aplica un efecto cascada para mejorar resultados. 
* TSA: es um módulo de fusion que agrega información comparando frames con sus anteriores y posteriores, toda esa información luego se usa en todos los frames.



Cambiar imágen 
!-[image.png](attachment:image.png)

# Pasos previos train

Como todavía no he comprobado como funciona no pongo nada, pero seria interesante hacer algo parecido al apartado siguiente

# Pasos pevios test

Lo primero es seleccionar los datasets o imágenes con las que se va a trabajar, si se quisiese usar un vídeo primero habría que generar la secuencia de frames y las imágenes en baja calidad.

Una vez hecho esto hay que crear un fichero .yml con las siguentes especificaciones:

```
name: EDVR_L_x4_REDS_SRblur_official
model_type: EDVRModel
scale: 4
num_gpu: 4 
manual_seed: 0

name:os cambiar el nombre que le queremos dar a la ejecución
model:el modelo siempre será el mismo ya que solo trabajamos con EDVR en BasicSR.
scale: es la escala de la salida respecto a la entrada, el upsampling ratio.
num_gpu: en todas nuestras ejecuciones el número de Gpu's es una por nustras limitaciones hardware, que se indica por 
         parametro.
```

```
datasets:
  test:
    name: REDS4
    type: VideoTestDataset
    dataroot_gt: datasets/REDS/train_sharp/train/train_sharp
    dataroot_lq: datasets/REDS/train_blur_bicubic/train/train_blur_bicubic/X4
    meta_info_file: basicsr/data/meta_info/meta_info_REDS4_test_GT.txt
    io_backend:
      type: disk

    cache_data: false
    num_frame: 5
    padding: replicate

name: el nombre del dataset.
type: es el tipo de dataset que se usa, depende de la estructura de los dataset, en nuestros casos se usan dos:
    video_test_dataset: 
                          dataroot
                          ├── subfolder1
                               ├── frame000
                               ├── frame001
                               ├── ...
                           ├── subfolder1
                               ├── frame000
                               ├── frame001
                               ├── ...
                           ├── ...
                           
    vimeo90k_dataset:  1. clip name; 2. frame number; 3. image shape, seperated by a white space.

dataroot_gt: path a las imágenes originales.
dataroot_lq: path a las imágenes en baja calidad.
meta_info_file: el fichero con la estructura de los datasets.
io_backend: Hay tres opciones para la ejecución disk, LMDB o memcached, solo se ha usdo disk para nustros experimentos.
padding: replicate o reflection_circle, son dos técnicas para conseguir que durante el procesamiento el tamaño de las 
         imágenes no cambie.
```

```
network_g:
  type: EDVR
  num_in_ch: 3
  num_out_ch: 3
  num_feat: 128
  num_frame: 5
  deformable_groups: 8
  num_extract_block: 5
  num_reconstruct_block: 40
  center_frame_idx: ~
  hr_in: false
  with_predeblur: true
  with_tsa: true

type: tipo de arquitectura que se usa, solo usaremos EDVR.
with_predeblur: tenemos la opción de activar o desactivar la opción de desemborronado previo. 
with_tsa: tenemos la opción de activar o desactivar la opción de TSA de fusion de frames, aunque es una de las                             funcionalidades básicas de EDVR.
```

```
path:
  pretrain_network_g: experiments/pretrained_models/EDVR/EDVR_L_x4_SRblur_REDS_official-983d7b8e.pth
  strict_load_g: true

pretrain_network_g: path al modelo preentrenado.
strict_load_g: tenemos la opción de usarlo o no.
```

```
val:
  save_img: true
  suffix: ~  
  
save_img: decide si se guardan las imágenes durante la validación.
```

 ```
 metrics:
    psnr: y
      type: calculate_psnr
      crop_border: 0
      test_y_channel: false

Metricas extra que calculan el PSNR(Peak Signal-to-Noise Ratio)
```

 # Datasets

Para las pruebas se han unsado 2 datasets, REDS dataset y Vimeo-90k dataset.

### REDS
Es el dataset oficial del NTIRE 2019 Y NTIRE 2020 Challenge, acronimo de REalistic and Dynamic Scenes esta centrado en el desemborronado y super resolucón de víedos. Tiene diferentes apartados con diferentes situaciones, imágens borrosas, borrosas y comprimidas, en baja resolución y borrosas y en baja resolución.

```
@InProceedings{Nah_2019_CVPR_Workshops_REDS,
  author = {Nah, Seungjun and Baik, Sungyong and Hong, Seokil and Moon, Gyeongsik and Son, Sanghyun and Timofte, Radu and Lee, Kyoung Mu},
  title = {NTIRE 2019 Challenge on Video Deblurring and Super-Resolution: Dataset and Study},
  booktitle = {The IEEE Conference on Computer Vision and Pattern Recognition (CVPR) Workshops},
  month = {June},
  year = {2019}
}
```
### Vimeo-90k
Dataset creado original mente para el proyecto Video Enhancement with Task-Oriented Flow, consistente en 89.800 clips de vídeo procedentes de vimeo.com, uno de sus principales funciones entre otra es servir para la super resolucion de vídeos. La versión que se utiliza consta de 91.701 secuencias de 7 frames con una resolución fija.


La cita perteece a el trabajo para el que se creo el dataset, no se si debería mencionarlo o no?¿
```
@article{xue2019video,
  title={Video Enhancement with Task-Oriented Flow},
  author={Xue, Tianfan and Chen, Baian and Wu, Jiajun and Wei, Donglai and Freeman, William T},
  journal={International Journal of Computer Vision (IJCV)},
  volume={127},
  number={8},
  pages={1106--1125},
  year={2019},
  publisher={Springer}
}
```

# Train and test:

Ambas opciones tienen varias opcioines de ejecución, aunque en nuestro caso solo se ha experimentado con una debeida a las limitaciones hardware.
La versión que se ha usado es la de una sola GPU, también se puede trabajar con múltiples GPU's(4 y 8), y por último  se puede trabajar en modo Slurm con 1, 4 o 8 GPU's.

La forma de ejecutar el script es my parecida, pero para las versiones multiGPU's hay que usar más parámetros opcionales.

La unica diferencia en este apartado respecto al el modo train o test se corresponde al path de archivos .yml que analizamos anteriormente.

### Train 1 GPU:

PYTHONPATH="./:${PYTHONPATH}" \
CUDA_VISIBLE_DEVICES=0 \
python basicsr/train.py -opt options/train/EDVR/train_EDVR_L_x4_SR_REDS.yml

Ningún archivo de train tiene cambiados los path!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


### Test 1 GPU:

PYTHONPATH="./:${PYTHONPATH}" \
CUDA_VISIBLE_DEVICES=0 \
python basicsr/test.py -opt options/test/EDVR/test_EDVR_L_x4_SRblur_REDS.yml

### Test 8 GPU's:
PYTHONPATH="./:${PYTHONPATH}" \
CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7 \
python -m torch.distributed.launch --nproc_per_node=8 --master_port=4321 basicsr/test.py -opt options/test/EDVR/test_EDVR_L_x4_SRblur_REDS.yml --launcher pytorch

### Test 1 GPU Slurm:
PYTHONPATH="./:${PYTHONPATH}" \
GLOG_vmodule=MemcachedClient=-1 \
srun -p [partition] --mpi=pmi2 --job-name=test --gres=gpu:1 --ntasks=1 --ntasks-per-node=1 --cpus-per-task=6 --kill-on-bad-exit=1 \
python -u basicsr/test.py -opt options/test/EDVR/test_EDVR_L_x4_SRblur_REDS.yml --launcher="slurm"

# Demostración y análisis de cómo funciona EDVR


A continuación se ejecutará una de las versiones del dataset REDS

In [1]:
!PYTHONPATH="./:${PYTHONPATH}"

In [2]:
!CUDA_VISIBLE_DEVICES=0

In [3]:
!python basicsr/test.py -opt options/test/EDVR/test_EDVR_L_x4_SRblur_REDS.yml

Disable distributed.
Path already exists. Rename it to /home/edvr/edvr/BasicSR/results/EDVR_L_x4_REDS_SRblur_official_archived_20210223_231639
2021-02-23 23:16:39,255 INFO: 
                ____                _       _____  ____
               / __ ) ____ _ _____ (_)_____/ ___/ / __ \
              / __  |/ __ `// ___// // ___/\__ \ / /_/ /
             / /_/ // /_/ /(__  )/ // /__ ___/ // _, _/
            /_____/ \__,_//____//_/ \___//____//_/ |_|
     ______                   __   __                 __      __
    / ____/____   ____   ____/ /  / /   __  __ _____ / /__   / /
   / / __ / __ \ / __ \ / __  /  / /   / / / // ___// //_/  / /
  / /_/ // /_/ // /_/ // /_/ /  / /___/ /_/ // /__ / /<    /_/
  \____/ \____/ \____/ \____/  /_____/\____/ \___//_/|_|  (_)
    
Version Information: 
	BasicSR: 1.2.0+1464d8e
	PyTorch: 1.7.0
	TorchVision: 0.8.1
2021-02-23 23:16:39,255 INFO: 
  name: EDVR_L_x4_REDS_SRblur_official
  model_type: EDVRModel
  scale: 4
  num_gpu: 4
  manual_seed: 0
  da

2021-02-23 23:16:45,810 INFO: Model [EDVRModel] is created.
2021-02-23 23:16:45,810 INFO: Testing REDS4...
2021-02-23 23:16:45,810 WARNING: nondist_validation is not implemented. Run dist_validation.
Test 020:100/100: 100%|████████████████████| 400/400 [03:32<00:00,  1.88frame/s]
2021-02-23 23:20:18,483 INFO: Validation REDS4
	 # psnr: 28.8819	 # 000: 27.3231	 # 011: 29.4919	 # 015: 31.2684	 # 020: 27.4444



Poner aqui cat de una o varias de las fotos resultado, baja resolucion y originales.